# Spark: Getting Started

## Step 0: Prerequisites & Installation

Follow [these instructions](https://docs.databricks.com/notebooks/notebooks-manage.html#import-a-notebook) to import this notebook into Databricks

Run these commands in your terminal (just once) if you want to run Spark locally.

 * These instructions require a Mac with [Anaconda3](https://anaconda.com/) and [Homebrew](https://brew.sh/) installed.
 * Useful for small data only. For larger data, try [Databricks](https://databricks.com/).

```bash
# Make Homebrew aware of old versions of casks
brew tap caskroom/versions

# Install Java 1.8 (OpenJDK 8)
brew cask install adoptopenjdk8

# Install the current version of Spark
brew install apache-spark

# Install Py4J (connects PySpark to the Java Virtual Machine)
pip install py4j

# Add JAVA_HOME to .bash_profile (makes Java 1.8 your default JVM)
echo "\n# Apache Spark\nexport JAVA_HOME=$(/usr/libexec/java_home -v 1.8)" >> ~/.bash_profile

# Add SPARK_HOME to .bash_profile
echo "\nexport SPARK_HOME=/usr/local/Cellar/apache-spark/2.4.3/libexec" >> ~/.bash_profile

# Add PySpark to PYTHONPATH in .bash_profile
echo "\nexport PYTHONPATH=$SPARK_HOME/python:$PYTHONPATH" >> ~/.bash_profile

# Update current environment
source ~/.bash_profile

```

## Step 1: Create a SparkSession with a SparkContext

In [ ]:
import pyspark

# un-comment the following lines if you are running Spark locally
# spark = pyspark.sql.SparkSession.builder.getOrCreate()
# sc = spark.sparkContext

In [ ]:
spark

In [ ]:
sc

If we need a broadcast (i.e. global) variable, we can declare it like so:

In [ ]:
glob = sc.broadcast(list(range(1, 3)))
glob.value

## Step 2: Download some Amazon reviews (Toys & Games)

In [ ]:
# Data is already in the repo, but you can also get it with
# !wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz
# !gunzip reviews_Toys_and_Games_5.json.gz

Follow [these instructions](https://docs.databricks.com/data/data.html#import-data-1) to import `reviews_Toys_and_Games_5.json` into Databricks

## Step 3: Create a Spark DataFrame

In [ ]:
# this file path will be different if you are running Spark locally
df = spark.read.json('/FileStore/tables/reviews_Toys_and_Games_5.json')

In [ ]:
df.persist()

This last command, `.persist()`, simply stores the DataFrame in memory. See [this page](https://unraveldata.com/to-cache-or-not-to-cache/). It is similar to `.cache()`, but actually more flexible than the latter since you can specify which storage level you want. See [here](https://stackoverflow.com/questions/26870537/what-is-the-difference-between-cache-and-persist).

In [ ]:
type(df)

In [ ]:
df.show(5) # default of 20 lines

In [ ]:
pdf = df.limit(5).toPandas()
pdf

In [ ]:
type(pdf)

In [ ]:
df.count()

In [ ]:
df.columns

In [ ]:
df.printSchema()

The 'nullable = true' bit means that the relevant column tolerates null values.

In [ ]:
df.describe().show()

In [ ]:
df.describe('overall').show()

In [ ]:
reviews_df = df[['asin', 'overall']]

In [ ]:
reviews_df.show()

In [ ]:
def show(df, n=5):
    return df.limit(n).toPandas()

In [ ]:
show(reviews_df)

In [ ]:
reviews_df.count()

In [ ]:
sorted_review_df = reviews_df.sort('overall')

In [ ]:
show(sorted_review_df)

In [ ]:
import pyspark.sql.functions as F

In [ ]:
counts = reviews_df.agg(F.countDistinct('overall'))

In [ ]:
counts.show()

In [ ]:
query = """
SELECT overall, COUNT(*)
FROM reviews
GROUP BY overall
ORDER BY overall
"""

In [ ]:
reviews_df.createOrReplaceTempView('reviews')

In [ ]:
output = spark.sql(query)

In [ ]:
show(output)

In [ ]:
output.collect()

In [ ]:
reviews_df.count() - sum(output.collect()[i][1] for i in range(5))

In [ ]:
type(reviews_df)

Convert to RDD!

In [ ]:
reviews_df.rdd

In [ ]:
type(reviews_df.rdd)

### Count the words in the first row

In [ ]:
row_one = df.first()

In [ ]:
row_one

In [ ]:
def word_count(text):
    return len(text.split())

In [ ]:
word_count(row_one['reviewText'])

In [ ]:
from pyspark.sql.types import IntegerType

#'udf' is for User Defined Function!

word_count_udf = F.udf(word_count, returnType=IntegerType())

In [ ]:
review_text_col = df['reviewText']

In [ ]:
counts_df = df.withColumn('wordCount', word_count_udf(review_text_col))

In [ ]:
# Remember that we set the default number of lines to show at 5.

show(counts_df).T

In [ ]:
from pyspark.sql.types import IntegerType
word_count_udf = F.udf(word_count, IntegerType())

# Registering our word_count() function so that we
# can use it with SQL! See documentation here:
# https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-UDFRegistration.html

df.createOrReplaceTempView('reviews')
spark.udf.register('word_count', word_count_udf)

In [ ]:
query = """
SELECT asin, overall, reviewText, word_count(reviewText) AS wordCount
FROM reviews
"""

In [ ]:
counts_df = spark.sql(query)

In [ ]:
show(counts_df)

In [ ]:
def count_all_the_things(text):
    return [len(text), len(text.split())]

In [ ]:
from pyspark.sql.types import ArrayType, IntegerType
count_udf = F.udf(count_all_the_things, returnType=ArrayType(IntegerType()))

In [ ]:
counts_df = df.withColumn('counts', count_udf(df['reviewText']))

In [ ]:
show(counts_df, 1)

In [ ]:
slim_counts_df = (
    df.drop('reviewTime', 'helpful')
#       .drop('helpful')
      .withColumn('counts', count_udf(df['reviewText']))
      .drop('reviewText')
)

In [ ]:
show(slim_counts_df, n=1)

In [ ]:
aggs = counts_df.groupBy('reviewerID').agg({'overall': 'mean'})
aggs.collect()

### A few more basic commands

Please refer also to the [official programming guide](http://spark.apache.org/docs/latest/rdd-programming-guide.html).

In [ ]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)

In [ ]:
distData.

In [ ]:
def multiply(a, b):
    return a * b

In [ ]:
distData.reduce(multiply)

In [ ]:
def subtract1(a, b):
    return a - b

In [ ]:
distData.reduce(subtract1)

In [ ]:
def subtract2(a, b):
    return b - a

In [ ]:
distData.reduce(subtract2)

Can you explain these "subtraction" results?

In [ ]:
distData.filter(lambda x: x < 4).collect()

### Reading files

```sc.textFile()``` for .txt files

`.toJSON()` for .json files

In [ ]:
dfjson = counts_df.toJSON()

In [ ]:
df2 = spark.read.json(dfjson)

In [ ]:
df2.printSchema()

In [ ]:
counts_df

In [ ]:
type(df.toPandas())